<a href="https://colab.research.google.com/github/tjtmddnjswkd/capstone/blob/jc/1_crawling/c2_crawling_hhn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
## 무조건 해야하는 부분 1
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb
CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
chmod +x /tmp/chromedriver
mv /tmp/chromedriver /usr/local/bin/chromedriver
pip install selenium

In [1]:
import pandas as pd
import re

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium import webdriver

# headers = {"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36"}

# 전체 과정 사이클

In [17]:
## colab

url = "https://elaw.klri.re.kr/kor_service/lawAllSearch.do?pCode=01"
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=chrome_options)

In [8]:
## local
try:
    save_path=r'G:\내 드라이브\KB인턴관련'
except:
    save_path='/content/drive/MyDrive/KB인턴관련'
url = "https://elaw.klri.re.kr/kor_service/lawAllSearch.do?pCode=01"
options = webdriver.ChromeOptions() # 크롬 옵션 객체 생성
options.add_argument('headless') # headless 모드 설정
options.add_argument("window-size=1920x1080") # 화면크기(전체화면)
options.add_argument("disable-gpu") 
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")

# 속도 향상을 위한 옵션 해제
prefs = {'profile.default_content_setting_values': {'cookies' : 2, 'images': 2, 'plugins' : 2, 'popups': 2, 'geolocation': 2, 'notifications' : 2, 'auto_select_certificate': 2, 'fullscreen' : 2, 'mouselock' : 2, 'mixed_script': 2, 'media_stream' : 2, 'media_stream_mic' : 2, 'media_stream_camera': 2, 'protocol_handlers' : 2, 'ppapi_broker' : 2, 'automatic_downloads': 2, 'midi_sysex' : 2, 'push_messaging' : 2, 'ssl_cert_decisions': 2, 'metro_switch_to_desktop' : 2, 'protected_media_identifier': 2, 'app_banner': 2, 'site_engagement' : 2, 'durable_storage' : 2}}   
options.add_experimental_option('prefs', prefs)

try:
    driver = webdriver.Chrome(r'C:\Users\yunjc\chromedriver_win32\chromedriver.exe', options=options)
except:
    driver = webdriver.Chrome(options=options)

C:\Users\yunjc\AppData\Local\Temp\ipykernel_31820\2420770139.py:15: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r'C:\Users\yunjc\chromedriver_win32\chromedriver.exe', options=options)


In [9]:
def wait_driver(xpath, t=10):
    WebDriverWait(driver, t).until(EC.presence_of_element_located((By.XPATH, xpath)))
    
def tab_move():
    driver.switch_to.window(driver.window_handles[-1])
    ko_url = driver.current_url.replace("ENG", "KOR")
    driver.get(ko_url)
    wait_driver('//*[@id="lawViewContent"]')
    driver.switch_to.frame("lawViewContent")

def tab_close():
    driver.close()
    driver.switch_to.window(driver.window_handles[0])
    driver.switch_to.frame("indexFrame")

def parsing(cate, law):
    contents = driver.find_elements(By.CSS_SELECTOR, '#body > div > table > tbody > tr > td')

    eng_line, kor_line = [], []
    for i in range(0, len(contents), 2):
        cur_eng, cur_kor = [], []
        eng_idx, kor_idx = i, i+1
        if contents[eng_idx].text=='' or contents[kor_idx].text=='':
            # print(contents[eng_idx].text)
            continue
        for key in ["none", "ho", "hang"]:
            cur_eng = list(map(lambda x: x.text, contents[eng_idx].find_elements(By.CLASS_NAME, key)))
            cur_kor = list(map(lambda x: x.text, contents[kor_idx].find_elements(By.CLASS_NAME, key)))
            if len(cur_eng)==len(cur_kor):
                eng_line+=cur_eng
                kor_line+=cur_kor
            else:
                return law
    return pd.DataFrame(zip([cate]*len(eng_line), [law]*len(kor_line), eng_line, kor_line), columns=["category", "lawName", "ENG", "KOR"])

In [ ]:
## crawler total

driver.get(url)
categories = [f'tree_{i}_a' for i in range(2, 47)]
# category loop
exception_list = []
base_df = pd.DataFrame()
for j, c in enumerate(categories):
    category = driver.find_element(By.ID, c)
    cate_list = category.text.split(" ")
    cate_name, num_of_laws = cate_list[1], int(cate_list[2].strip('()'))
    driver.execute_script("arguments[0].click();", category)

    # pages
    driver.switch_to.frame('indexFrame')
    # pages = driver.find_elements(By.CSS_SELECTOR, "#container > div > div.paging > a")
    pages = [f'//*[@id="container"]/div/div[3]/a[{k}]' for k in range(3, 13)]
    page_idx = 0
    complete_num = 0
    print(f"{j+1}. {cate_name} 시작...")
    df_laws = pd.DataFrame()
    while True:
    # laws loop
        wait_driver('//*[@id="1"]/td[2]')
        laws = driver.find_elements(By.CSS_SELECTOR, '#list1 > tbody > tr > td > p > a')
        page_parsing_num = 0
        page_exception_num = 0
        for law in laws:
            law_name = law.text.split('\n')[0]
            law.send_keys(Keys.ENTER)

            tab_move()
            wait_driver('/html')

            # Parsing
            result = parsing(cate_name, law_name)
            # fininsh parsing
            tab_close()
            if type(result)==str:
                exception_list.append(result)
                page_exception_num+=1
            else:
                df_laws = pd.concat([df_laws, result], axis=0)
                page_parsing_num+=1
        print(f"\t{page_idx+1}페이지 파싱 완료: {page_parsing_num}\t예외처리: {page_exception_num}")
        complete_num += len(laws)
        if complete_num < num_of_laws:
            driver.find_element(By.XPATH, pages[page_idx]).click()
            page_idx+=1
        else:
            break
    print(f"{cate_name} 파싱 완료\n------------------------------------------")
    df_laws.to_csv(save_path+f"/{cate_name}.csv", index=False, encoding='euc-kr')
    base_df = pd.concat([base_df, df_laws], axis=0)
    driver.get(url)
    wait_driver('//*[@id="tree_1"]')
driver.quit()

In [12]:
import pickle

with open(save_path+'/execption_list.pkl', 'wb') as f:
    pickle.dump(exception_list, f)

exception_list

['겨레말큰사전남북공동편찬사업회법',
 '국가장법',
 '남북교류협력에 관한 법률',
 '남북협력기금법',
 '대한민국헌법',
 '법령 등 공포에 관한 법률',
 '북한이탈주민의 보호 및 정착지원에 관한 법률',
 '북한이탈주민의 보호 및 정착지원에 관한 법률 시행령',
 '상훈법',
 '생활물류서비스산업발전법 시행령',
 '영해 및 접속수역법',
 '영해및접속수역법시행령',
 '국회에서의 증언·감정 등에 관한 법률',
 '남북관계 발전에 관한 법률 시행령',
 '대한민국헌정회 육성법',
 '인사청문회법',
 '공공단체등 위탁선거에 관한 법률',
 '공직선거법',
 '국민투표법',
 '선거관리위원회법',
 '정당법',
 '개인정보 보호법',
 '공공기관의 정보공개에 관한 법률',
 '공공기관의 정보공개에 관한 법률 시행령',
 '공공기록물 관리에 관한 법률',
 '공공데이터의 제공 및 이용 활성화에 관한 법률',
 '공공데이터의 제공 및 이용 활성화에 관한 법률 시행령',
 '공익신고자 보호법',
 '국가정보원법',
 '대통령 등의 경호에 관한 법률',
 '대통령기록물 관리에 관한 법률 시행령',
 '대통령직 인수에 관한 법률',
 '민원 처리에 관한 법률',
 '법제업무 운영규정',
 '온실가스 배출권의 할당 및 거래에 관한 법률',
 '전자정부법',
 '정부조직법',
 '정부출연연구기관 등의 설립ㆍ운영 및 육성에 관한 법률',
 '책임운영기관의 설치·운영에 관한 법률',
 '행정규제기본법',
 '행정규제기본법 시행령',
 '행정기관 소속 위원회의 설치·운영에 관한 법률',
 '행정대집행법',
 '고위공무원단 인사규정',
 '공무원 인재개발법 시행령',
 '공무원 재해보상법',
 '공무원 징계령',
 '공무원 행동강령',
 '공무원수당 등에 관한 규정',
 '공무원연금법',
 '공무원연금법 시행령',
 '공무원임용령',
 '공무원임용시험령',
 '공직자윤리법',
 '공직자윤리법 시행령',
 '국가공무원 복무규정',
 '국가공무원법',
 '법관의 보수에 관한 

In [13]:
base_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 220521 entries, 0 to 20
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   category  220521 non-null  object
 1   lawName   220521 non-null  object
 2   ENG       220521 non-null  object
 3   KOR       220521 non-null  object
dtypes: object(4)
memory usage: 8.4+ MB


In [14]:
base_df.to_csv(save_path+'/crawl_total.csv', index=False, encoding='euc-kr')

In [16]:
print(f"파싱 법령 수:{base_df.lawName.nunique()}\t예외 법령 수: {len(exception_list)}\t법령총합:{base_df.lawName.nunique()+len(exception_list)}\t총 법령 수: 2341")

파싱 법령 수:1135	예외 법령 수: 1193	법령총합:2328	총 법령 수: 2341
